In [1]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_api_core-2.19.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached googleapis_common_protos-1.63.2-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached proto_plus-1.24.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB 991.0 kB/s eta 0:00:13
    --------------------------------------- 0.3/12.0 MB 3.2 MB/s eta 0:00:04
   - -------------------------------------- 0.4/12.0 MB 3.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/12.

In [35]:
import os
import io
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

In [4]:
path_to_cred = "./kai-ai-nano-dbb2086d4c36.json" # this is a Credentials from Google cloud concole, need to be dowloaded
os.path.isfile(path_to_cred)

True

In [7]:
local_file = "./test_file/test_1.pdf"
os.path.isfile(local_file)

True

In [14]:
creds = service_account.Credentials.from_service_account_file(path_to_cred,
                                                                  scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)
GD_folder_id = "10sHUaht5ibGzjBFKHYfcuBguiUrp27IX"
# to get GD_folder_id, share the folder and looking for URL after folders/ and before ?usp=sharing

In [24]:
#10sHUaht5ibGzjBFKHYfcuBguiUrp27IX
def upload_file_to_GD(local_file, file_name, GD_folder_id): 
    file_metadata = {
    'name': file_name,  
    'parents': [GD_folder_id]}  # ID of the folder where you want to upload
    #media = MediaFileUpload(local_file, mimetype='application/pdf')
    media = MediaFileUpload(local_file)
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

In [16]:
def create_temp_folder(GD_folder_name): 
    folder_metadata = {
    'name': GD_folder_name,  
    'parents': ['10sHUaht5ibGzjBFKHYfcuBguiUrp27IX']}  # ID of the parent folder (optional)
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()

In [13]:
#create_temp_folder("new_fol")

In [25]:
upload_file_to_GD(local_file, "test_1", GD_folder_id)

In [28]:
def list_files_in_folder(folder_id):
    query = f"'{folder_id}' in parents"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    items = results.get('files', [])
    return items

In [30]:
list_files_in_folder(GD_folder_id)

[{'id': '1zlUAp7YVYWIqwyWBgtgQYksdLE020MDv', 'name': 'test_1'}]

In [31]:
def download_file(file_id, file_name, destination_folder):
    request = drive_service.files().get_media(fileId=file_id)
    file_path = os.path.join(destination_folder, file_name)
    fh = io.FileIO(file_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"Download {file_name} {int(status.progress() * 100)}%.")

In [37]:
download_file('1zlUAp7YVYWIqwyWBgtgQYksdLE020MDv', "download_test.pdf", "./test_download")

Download download_test.pdf 100%.
